In [3]:
#importing necessary libraries
import pandas as pd
import sqlite3

#Establishing connection to database
conn = sqlite3.connect("pharm_data.db")

#Fetching list of tables we have access to, for reference
cursor = conn.cursor()
tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("ALL TABLES: ",end="")
for table in tables: print(table[0],end=", ")
print("\b\b")

#Asks user for their zip code, gets state, determines pdp_region_code based on state
user_zip = input("Enter your zipcode: ")
user_state = pd.read_sql("select COUNTY,STATENAME\
                          from ZIP_STATE_COUNTY\
                          where ZIPCODE is '" + user_zip + "'", conn)\
            ['STATENAME'].values[0].rstrip()

user_pdp_region_code = pd.read_sql("select distinct PDP_REGION_CODE\
                                    from GEOGRAPHIC_LOCATOR\
                                    where STATENAME is '"+user_state+"'",conn)\
                       ['PDP_REGION_CODE'].values[0]

ALL TABLES: BENEFICIARY_COST, GEOGRAPHIC_LOCATOR, PLAN_INFO, ZIP_STATE_COUNTY, NCPDP, FORMULARY_DATA, NDC_DATA, RXCUI_DATA
Enter your zipcode: 07095


In [4]:
#Step 3, filtering formulary_ids by pdp_region_code and contract_id
F_SUBSET = pd.read_sql("select * from PLAN_INFO\
                        where PDP_REGION_CODE is '"+str(user_pdp_region_code)+"'\
                        and CONTRACT_ID like 'S%'",conn)\
           [['CONTRACT_ID','PLAN_ID','SEGMENT_ID','FORMULARY_ID','PREMIUM','DEDUCTIBLE','PDP_REGION_CODE']]
F_SUBSET

,CONTRACT_ID,PLAN_ID,SEGMENT_ID,FORMULARY_ID,PREMIUM,DEDUCTIBLE,PDP_REGION_CODE
0,S0522,4,0,18041,30.4,405.0,4
1,S4802,78,0,18443,36.9,405.0,4
2,S4802,101,0,18439,79.5,0.0,4
3,S5601,8,0,18419,34.3,0.0,4
4,S5601,9,0,18420,84.6,0.0,4
5,S5617,18,0,18082,74.8,405.0,4
6,S5617,249,0,18083,52.9,0.0,4
7,S5660,106,0,18154,34.0,405.0,4
8,S5660,207,0,18155,91.7,350.0,4
9,S5660,220,0,18152,22.6,405.0,4


In [8]:
#Step 4, asking user for drugs and strengths
user_generics = {(input("Enter a generic drug name: ").upper(),\
                  input("Enter a desired strength: ")):[]\
                 for i in range(int(input("Enter how many drugs you wish to search: ")))}
user_generics

Enter how many drugs you wish to search: 2
Enter a generic drug name: Lisinopril
Enter a desired strength: 10
Enter a generic drug name: Somatropin
Enter a desired strength: 0


{('LISINOPRIL', '10'): [], ('SOMATROPIN', '0'): []}

In [ ]:
#getting NDCs for each drug,strength combo the user offered in step 4
# for drug,strength in user_generics:
#     user_generics[(drug,strength)] = pd.read_sql("select PRODUCTNDC from NDC_DATA\
#                  where upper(NONPROPRIETARYNAME) like '%"+drug+"%'\
#                  and (ACTIVE_NUMERATOR_STRENGTH is '' or ACTIVE_NUMERATOR_STRENGTH is "+str(strength)+")",conn)\
#     .applymap(lambda s: "0"*(11-(len(s)-1))+s.replace("-",""))

In [ ]:
# for drug,strength in user_generics:
#     print("DRUG: %s, STRENGTH: %s\n%s" % (drug,strength,user_generics[drug,strength]))

In [ ]:
# drug = 'LISINOPRIL'
# strength = 10
# pd.read_sql("select * from FORMULARY_DATA\
#  where NDC in (select PRODUCTNDC from NDC_DATA\
#                  where upper(NONPROPRIETARYNAME) like '%"+drug+"%'\
#                  and (ACTIVE_NUMERATOR_STRENGTH is '' or ACTIVE_NUMERATOR_STRENGTH is "+str(strength)+"))",conn)
    

In [ ]:
# pd.read_sql("select * from FORMULARY_DATA",conn)

In [ ]:
# CLEAN_NDC_DATA_TABLE = pd.read_sql("select PRODUCTNDC from NDC_DATA",conn).applymap(lambda s: s.replace("-","")).iloc[1:]
# def clean_ndc(s):
#     if len(s) in [11,10,9]:
        
# CLEAN_NDC_FD_TABLE = pd.read_sql("select NDC from FORMULARY_DATA",conn).applymap()
# pd.read_sql("select PRODUCTNDC from CLEAN_NDC_DATA_TABLE where PRODUCTNDC is '00060106'",conn)

In [ ]:
#RXCUI REWORK
# pd.read_sql("select distinct RXN_DOSE_FORM from RXCUI_DATA",conn)

In [17]:
for drug,strength in user_generics:
    user_generics[(drug,strength)] = \
    pd.read_sql("select FORMULARY_DATA.* \
from (PLAN_INFO join FORMULARY_DATA \
      on instr(FORMULARY_DATA.FORMULARY_ID, PLAN_INFO.FORMULARY_ID) > 0)\
      where PLAN_INFO.PDP_REGION_CODE is '"+str(user_pdp_region_code)+"' \
      and PLAN_INFO.CONTRACT_ID like 'S%' \
      and FORMULARY_DATA.RXCUI in (select RXCUI_DATA.RXCUI from RXCUI_DATA\
    where upper(RXCUI_DATA.FULL_GENERIC_NAME) like '%"+drug+"%' and\
    RXCUI_DATA.STRENGTH like '%"+strength+" %')",conn)

In [18]:
user_generics[('LISINOPRIL','10')]

,FORMULARY_ID,FORMULARY_VERSION,CONTRACT_YEAR,RXCUI,NDC,TIER_LEVEL_VALUE,QUANTITY_LIMIT_YN,QUANTITY_LIMIT_AMOUNT,QUANTITY_LIMIT_DAYS,PRIOR_AUTHORIZATION_YN,STEP_THERAPY_YN
0,00018039,9,2018,314076,43547035310,1,Y,60,30,N,N
1,00018039,9,2018,197885,68180051802,1,Y,30,30,N,N
2,00018040,8,2018,314076,43547035310,1,Y,60,30,N,N
3,00018040,8,2018,197885,68180051802,1,Y,30,30,N,N
4,00018041,8,2018,314076,43547035310,1,Y,60,30,N,N
5,00018041,8,2018,197885,68180051802,1,Y,30,30,N,N
6,00018043,8,2018,314076,43547035310,1,Y,60,30,N,N
7,00018043,8,2018,197885,68180051802,1,Y,30,30,N,N
8,00018056,8,2018,314076,43547035310,1,N,,,N,N
9,00018056,8,2018,197885,68180051802,1,N,,,N,N
